In [2]:
from lerobot.model.kinematics import RobotKinematics

In [3]:
#!/usr/bin/env python3

import numpy as np
import placo
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_robot_pose(robot, ax):
    """可视化机器人当前姿态"""
    # 获取所有link的位置
    links = []
    for link_name in robot.link_names():
        T = robot.get_T_world_frame(link_name)
        links.append(T[:3, 3])  # 只取位置信息
    links = np.array(links)
    
    # 绘制机器人骨架
    ax.plot(links[:, 0], links[:, 1], links[:, 2], 'b-', linewidth=2)
    ax.scatter(links[:, 0], links[:, 1], links[:, 2], c='r', marker='o')

def main():
    # 配置参数
    urdf_path = "/home/xlk/work/lerobot/examples/hil-serl/urdfs/aloha_new_v00.urdf"
    target_frame = "gripper_frame_link"
    joint_names = [
        "fl_joint1", "fl_joint2", "fl_joint3",
        "fl_joint4", "fl_joint5", "fl_joint6",
        "gripper_frame_joint"
    ]

    # 1. 初始化机器人和求解器
    robot = placo.RobotWrapper(urdf_path)
    solver = placo.KinematicsSolver(robot)
    solver.mask_fbase(True)  # 固定基座

    # 2. 设置初始关节角度（以度为单位）
    initial_joints = np.array([0.0, -45.0, 0.0, 90.0, 0.0, 0.0, 0.0])
    initial_joints_rad = np.deg2rad(initial_joints)

    # 更新机器人关节位置
    for i, name in enumerate(joint_names):
        robot.set_joint(name, initial_joints_rad[i])
    robot.update_kinematics()

    # 3. 计算正运动学
    T_initial = robot.get_T_world_frame(target_frame)
    print("初始末端执行器位姿:\n", T_initial)

    # 4. 设置目标位姿（在当前位置基础上偏移）
    target_pose = T_initial.copy()
    target_pose[0:3, 3] += np.array([0.1, 0.1, 0.1])  # 在xyz方向各偏移10cm

    # 5. 设置IK任务
    tip_frame = solver.add_frame_task(target_frame, target_pose)
    tip_frame.configure(target_frame, "soft", 1.0, 0.01)  # 位置权重1.0，姿态权重0.01

    # 6. 求解IK
    solver.solve(True)
    robot.update_kinematics()

    # 7. 获取求解结果
    final_joints = []
    for name in joint_names:
        joint = robot.get_joint(name)
        final_joints.append(np.rad2deg(joint))
    final_joints = np.array(final_joints)

    print("\n求解得到的关节角度（度）:", final_joints)
    
    # 8. 验证结果
    T_final = robot.get_T_world_frame(target_frame)
    print("\n最终末端执行器位姿:\n", T_final)
    
    # 计算位置误差
    pos_error = np.linalg.norm(T_final[:3, 3] - target_pose[:3, 3])
    print(f"\n位置误差: {pos_error*1000:.2f}mm")

    # 9. 可视化（可选）
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # 画出初始和最终位置
    ax.scatter(T_initial[0, 3], T_initial[1, 3], T_initial[2, 3], 
              c='g', marker='o', s=100, label='Initial Position')
    ax.scatter(target_pose[0, 3], target_pose[1, 3], target_pose[2, 3], 
              c='r', marker='*', s=100, label='Target Position')
    ax.scatter(T_final[0, 3], T_final[1, 3], T_final[2, 3], 
              c='b', marker='^', s=100, label='Final Position')
    
    # 画出机器人当前姿态
    visualize_robot_pose(robot, ax)
    
    # 设置图表属性
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.legend()
    
    # 设置视角
    ax.view_init(elev=30, azim=45)
    
    plt.show()

if __name__ == "__main__":
    main()

ImportError: libboost_python38.so.1.71.0: cannot open shared object file: No such file or directory